In [1]:
# %matplotlib notebook 
%matplotlib inline

from my_functions import *

import time
import numpy as np
import matplotlib.pyplot as plt

import dynamic_graph_manager_cpp_bindings
from robot_properties_solo.solo8wrapper import Solo8Config 

In [ ]:
# Create the dgm communication to the control process.
head = dynamic_graph_manager_cpp_bindings.DGMHead(Solo8Config.dgm_yaml_path)

K = 5
D = 0.1
dt = 0.001
next_time = time.time() + dt


stored_P = []
stored_V = []
stored_t = []
stored_C = []


slider_P = head.get_sensor('slider_positions')
joint_P = head.get_sensor('joint_positions')
joint_V = head.get_sensor('joint_velocities')
contact = head.get_sensor('contact_sensors')

while True:
    if time.time() >= next_time:
        next_time += dt
        
        head.read()
        
        if slider_P[1] > 0.5 or contact[0] < 0.5:
            head.set_control('ctrl_joint_torques', np.zeros(8))
            head.write()
            break

#         tau = K*(target - joint_P) - D*joint_V
        tau = np.zeros(8)

        # write
        head.set_control('ctrl_joint_torques', tau)
        head.write()
        
#         stored_P.append(np.array(joint_P, copy=True))
#         stored_V.append(np.array(joint_V, copy=True))
#         stored_t.append(next_time)
        stored_C.append(np.array(contact, copy=True))

    time.sleep(0.0001)

In [4]:
head = dynamic_graph_manager_cpp_bindings.DGMHead(Solo8Config.dgm_yaml_path)

K = 5
D = 0.1
dt = 0.001
next_time = time.time() + dt


###
leg_down = 0.1
leg_up = 0.25
extend = -0.015  # extension constant

Targets = (
    get_target(leg_down-extend, leg_down, leg_up, leg_up, leg_up, leg_up, leg_down+extend, leg_down),
    get_target(leg_up, leg_up, leg_down-extend, leg_down, leg_down+extend, leg_down, leg_up, leg_up),
)


L = None
i_Targets = 0
i_L = 0


slider_P = head.get_sensor('slider_positions')
joint_P = head.get_sensor('joint_positions')
joint_V = head.get_sensor('joint_velocities')
contact = head.get_sensor('contact_sensors')


# In your control loop:
while True:
    if time.time() >= next_time:
        next_time += dt

        # read
        head.read()
        
        if slider_P[0] > 0.5:
            head.set_control('ctrl_joint_torques', np.zeros(8))
            head.write()
            break

        if L is None or i_L >= len(L):
            L = np.linspace(joint_P, Targets[i_Targets], num=250)
            i_L = 0
            i_Targets = (i_Targets + 1) % len(Targets)


        target = L[i_L]
        i_L += 1

        tau = K*(target - joint_P) - D*joint_V

        # write
        head.set_control('ctrl_joint_torques', tau)
        head.write()

    time.sleep(0.0001)

In [ ]:
stored_P = np.array(stored_P)
stored_V = np.array(stored_V)
stored_t = np.array(stored_t)


slice_data = slice(2000,7500)
print(stored_P.shape)
print(stored_t.shape)

In [ ]:
stored_C = np.array(stored_C)

In [ ]:
plt.plot(stored_C[:, 0])

In [ ]:
plt.plot(stored_P[slice_data, 2])

In [ ]:
plt.plot(
    np.gradient(
        stored_P[slice_data, 2]
        
    )
)

In [ ]:
plt.plot(stored_V[slice_data, 2])

In [ ]:
plt.plot(
    np.gradient(
        stored_V[slice_data,2]
    )
)

# slider

In [3]:
from my_functions import *

import time
import numpy as np
import matplotlib.pyplot as plt

import dynamic_graph_manager_cpp_bindings
from robot_properties_solo.solo8wrapper import Solo8Config 

# Create the dgm communication to the control process.
head = dynamic_graph_manager_cpp_bindings.DGMHead(Solo8Config.dgm_yaml_path)


D = 0.05
dt = 0.001
next_time = time.time() + dt

K = 5

angle_adjust = np.array([1,-2,1,-2,-1,2,-1,2]) * np.pi


# In your control loop:
# head.read()
slider_P = head.get_sensor('slider_positions')
joint_P = head.get_sensor('joint_positions')
joint_V = head.get_sensor('joint_velocities')
contact = head.get_sensor('contact_sensors')


while (True):
    if time.time() >= next_time:
        next_time += dt

        head.read()
        
        if slider_P[1] > 0.5 or contact[0] < 0.5:  # exit
            print('slider or contact break')
            head.set_control('ctrl_joint_torques', np.zeros(8))
            head.write()
            break
        
        if slider_P[0]/2 *np.pi > 2.7:
            tau = np.zeros(8)
        else:
            target = slider_P[0]/2 * angle_adjust
            target[2:] = np.zeros(6)

            tau = K*(target - joint_P) - D*joint_V

        head.set_control('ctrl_joint_torques', tau)
        head.write()

    time.sleep(0.0001)
    
print(slider_P, contact)

KeyboardInterrupt: 